In [4]:
import gurobipy as gp
import pandas as pd

# Create a new model
model = Model("Job_Scheduling")
file_path = "/Users/sophiehuang/Documents/112-2Operation Research/OR midtern project/instance05.csv"
data = pd.read_csv(file_path)

# Extracting data from dataframe
jobs = data['Job ID'].tolist()
due_times = data['Due Time'].tolist()
processing_times_stage1 = data['Stage-1 Processing Time'].tolist()
processing_times_stage2 = data['Stage-2 Processing Time'].tolist()

# fill N/A
data['Stage-1 Machines'].fillna('0', inplace=True)
data['Stage-2 Machines'].fillna('0', inplace=True)

# make sure all of them is string
data['Stage-1 Machines'] = data['Stage-1 Machines'].astype(str)
data['Stage-2 Machines'] = data['Stage-2 Machines'].astype(str)

# Extract machines as sets for each job and stage
stage1_machines = [set(map(int, x.split(','))) for x in data['Stage-1 Machines']]
stage2_machines = [set(map(int, x.split(','))) for x in data['Stage-2 Machines']]
all_machines = set.union(*stage1_machines, *stage2_machines)

# Define variables
c = model.addVars(jobs, [1, 2], name="completion_time", vtype=GRB.CONTINUOUS)
t = model.addVars(jobs, name="tardiness", vtype=GRB.CONTINUOUS)
x = model.addVars(jobs, [1, 2], all_machines, name="machine_assignment", vtype=GRB.BINARY)
y = model.addVars(jobs, [1, 2], jobs, [1, 2], all_machines, name="job_precedence", vtype=GRB.BINARY)

# Objective: minimize the total tardiness
model.setObjective(quicksum(t[j] for j in jobs), GRB.MINIMIZE)

# Constraints
# Tardiness and completion time constraints
for j in jobs:
    model.addConstr(t[j] >= c[j, 2] - due_times[j - 1], name=f"tardiness_{j}")
    model.addConstr(t[j] >= 0, name=f"non_neg_tardiness_{j}")
    model.addConstr(c[j, 2] - c[j, 1] - processing_times_stage2[j - 1] <= 1, name=f"stage_gap_ub_{j}")
    model.addConstr(c[j, 2] - c[j, 1] - processing_times_stage2[j - 1] >= 0, name=f"stage_gap_lb_{j}")

# Machine assignment constraints
for j in jobs:
    for k in [1, 2]:
        model.addConstr(quicksum(x[j, k, i] for i in all_machines if i in (stage1_machines[j - 1] if k == 1 else stage2_machines[j - 1])) == 1,
                        name=f"machine_assignment_{j}_{k}")

# Job precedence constraints
L = 1000  # A large number
for j in jobs:
    for k in [1, 2]:
        for l in jobs:
            if j != l:
                for h in [1, 2]:
                    for i in all_machines:
                        if i in stage1_machines[j - 1] and i in stage1_machines[l - 1]:
                            model.addConstr(c[j, k] + processing_times_stage1[l - 1] - c[l, h] <= L * (1 - y[j, k, l, h, i]),
                                            name=f"precedence_{j}_{k}_{l}_{h}_{i}")

# Add constraints for completion times to be non-negative
for j in jobs:
    for k in [1, 2]:
        model.addConstr(c[j, k] >= 0, name=f"non_neg_completion_{j}_{k}")
        
# Constraint: x_jki = 0 for all i not in M_jk
for j in jobs:
    for k in [1, 2]:
        unavailable_machines = all_machines - (stage1_machines[j - 1] if k == 1 else stage2_machines[j - 1])
        for i in unavailable_machines:
            model.addConstr(x[j, k, i] == 0, name=f"invalid_machine_assignment_{j}_{k}_{i}")

# Job scheduling constraints (assuming corrections to use y variables)
for i in all_machines:
    for j in jobs:
        for k in [1, 2]:
            for l in jobs:
                if j != l:
                    for h in [1, 2]:
                        model.addConstr(x[j, k, i] + x[l, h, i] <= y[j, k, l, h, i] + y[l, h, j, k, i] + 1, name=f"job_scheduling_{j}_{k}_{l}_{h}_{i}")


# Optimize the model
model.optimize()

# Retrieve and print solution if exists
if model.status == GRB.OPTIMAL:
    solution = {v.varName: v.x for v in model.getVars()}
    print("Optimal solution found.")
else:
    print("No optimal solution found.")
    solution = None

solution


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.4.0 21E230)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information